# Pacote

In [4]:
import pandas as pd
import chardet
import glob
from unidecode import unidecode
import os
import re
pd.set_option('display.max_columns', None)

# Funções

In [7]:
def process_file(file_path):
    """
    Carrega e transforma dados meteorológicos de um arquivo CSV em uma estrutura de DataFrame requerida.
    
    O arquivo CSV é processado para incluir informações da estação e converter coordenadas e altitude para float.

    Parâmetros:
        file_path (str): Caminho completo para o arquivo CSV a ser processado.
    
    Retorna:
        DataFrame: Um DataFrame do pandas com os dados da estação meteorológica processados.
    """
    
    # Carregar os dados pulando as primeiras 8 linhas que contêm informações da estação
    df = pd.read_csv(file_path, delimiter=';', encoding='ISO-8859-1', skiprows=8)
    
    # Carregar as informações da estação
    station_info_df = pd.read_csv(file_path, delimiter=';', encoding='ISO-8859-1', nrows=8, header=None)
    station_info_df.columns = ['Variable', 'Value']
    station_info_dict = station_info_df.set_index('Variable')['Value'].to_dict()

     # Converter latitude, longitude e altitude para float, substituindo vírgula por ponto
    for key in ['LATITUDE:', 'LONGITUDE:', 'ALTITUDE:']:
        if key in station_info_dict and isinstance(station_info_dict[key], str):
            try:
                # Tentativa de converter o valor para float após substituir vírgula por ponto
                station_info_dict[key] = float(station_info_dict[key].replace(',', '.'))
            except ValueError:
                # Tratar o erro de conversão, por exemplo, definindo o valor como None ou um valor padrão
                station_info_dict[key] = None  # ou algum valor padrão, se apropriado

    # Adicionar informações da estação como novas colunas ao DataFrame
    for variable, value in station_info_dict.items():
        # Removendo os dois pontos do final do nome da variável, se houver
        df[variable.rstrip(':')] = value
    
    return df

def detect_encoding(file_pattern_or_path, num_bytes=10000):
    """
    Detecta a codificação do arquivo ou arquivos fornecidos.
    
    Parâmetros:
        file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
        num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
    
    Retorna:
        dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
    """
    
    # Encontrar arquivos que correspondem ao padrão ou caminho fornecido.
    files = glob.glob(file_pattern_or_path)
    encodings = {}

    # Iterar sobre cada arquivo encontrado.
    for file_path in files:
        # Abrir arquivo em modo binário e ler os primeiros 'num_bytes' bytes.
        with open(file_path, 'rb') as f:
            rawdata = f.read(num_bytes)
            # Detectar a codificação do fragmento lido e armazenar no dicionário.
            encodings[file_path] = chardet.detect(rawdata)["encoding"]
    
    return encodings


def padronizar_nome_coluna(coluna):
    """
    Padroniza o nome de uma coluna removendo acentos, transformando em minúsculas e substituindo espaços, hífens, 
    parênteses, pontos, vírgulas, porcentagens e barras por underscores ou removendo-os. Remove underscores duplicados.

    Parâmetros:
    coluna (str): Nome da coluna a ser padronizado.

    Retorna:
    str: Nome da coluna padronizado.
    """
    from unidecode import unidecode

    # Remover acentos do nome da coluna.
    coluna = unidecode(coluna)
    # Transformar todas as letras em minúsculas.
    coluna = coluna.lower()
    # Substituir espaços, hífens, parênteses, pontos, vírgulas, porcentagens, barras e barras invertidas por underscores ou removendo-os.
    coluna = coluna.replace(' ', '_').replace('-', '_').replace('(', '_').replace(')', '_').replace('.', '_').replace(',', '_').replace('%', 'pcnt').replace('/', '').replace('\\', '')
    # Substituir underscores duplicados por um único underscore.
    while '__' in coluna:
        coluna = coluna.replace('__', '_')

    return coluna

def dms_to_decimal(dms):
    """
    Converte coordenadas do formato DMS (graus, minutos, segundos) para graus decimais.
    
    Parâmetros:
        dms (str): Uma string representando as coordenadas em formato DMS.
    
    Retorna:
        float: O valor decimal das coordenadas fornecidas.
    
    Exemplos de formato DMS:
        - "05°56'00.0\"S" -> -5.933333
        - "040°17'51.0\"W" -> -40.297500
    """

    # Encontrar os números (graus, minutos, segundos) e a direção (N/S/E/W)
    numbers = re.findall(r'\d+', dms)
    direction = re.search(r'[NSEW]', dms).group()
    
    # Converter graus, minutos e segundos para float e calcular o valor decimal
    decimal = float(numbers[0]) + float(numbers[1])/60 + float(numbers[2])/3600
    
    # Se a direção for Sul ou Oeste, o valor deve ser negativo
    if direction in ['S', 'W']:
        decimal *= -1
    return decimal

def clean_name_aero(name):
    """
    Limpa e padroniza nomes de aeródromos removendo conteúdos entre parênteses,
    acentos, caracteres especiais e convertendo para maiúsculas.

    Args:
    name (str): Nome do aeródromo a ser limpo.

    Returns:
    Series: Nome do aeródromo limpo e padronizado.
    """

    # Remove o conteúdo dentro de parênteses.
    # Utiliza expressões regulares para identificar texto entre parênteses e substituir por string vazia.
    # A função strip() é usada para remover espaços em branco no início e no fim da string.
    name = pd.Series(name).replace(r'\(.*?\)', '', regex=True).str.strip()

    # Remove acentos e caracteres especiais.
    # Normaliza a string para a forma de decomposição NFKD, que separa letras de acentos.
    # Codifica a string em ASCII, ignorando erros (ignorando caracteres não ASCII).
    # Decodifica a string de volta para UTF-8.
    # Converte a string para letras maiúsculas.
    return name.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8').str.upper()

# Carregando dados

## Meteorologicos

In [91]:
# The base path where the CSV files are located
base_path = 'DADOS_METEROLOGIA/CLIMA_2023/*.CSV'

# Initialize an empty list to collect all dataframes
all_dataframes = []

# Get a list of all CSV file paths
file_paths = glob.glob(base_path)

# Process each file and append the result to the list
for file_path in file_paths:
    df_processed = process_file(file_path)
    all_dataframes.append(df_processed)

In [92]:
# Concatenate all dataframes into one
final_dataframe = pd.concat(all_dataframes, ignore_index=True)

# Renomear colunas do DataFrame
colunas_renomeadas = {
     list(final_dataframe)[0]: 'data',
    list(final_dataframe)[1]: 'hora_utc',
    'REGI?O': 'REGIAO',
    'ESTAC?O': 'ESTACAO',
    'DATA DE FUNDAC?O': 'DATA DE FUNDACAO'
}

final_dataframe.rename(columns=colunas_renomeadas, inplace=True)

In [93]:
# Aplicar a função a cada nome de coluna
final_dataframe.columns = [padronizar_nome_coluna(col) for col in final_dataframe.columns]

In [94]:
final_dataframe = final_dataframe[['data', 'hora_utc',
 'precipitacao_total_horario_mm_', 'pressao_atmosferica_ao_nivel_da_estacao_horaria_mb_',
 'pressao_atmosferica_max_na_hora_ant_aut_mb_', 'pressao_atmosferica_min_na_hora_ant_aut_mb_',
 'temperatura_do_ar_bulbo_seco_horaria_degc_', 'temperatura_do_ponto_de_orvalho_degc_',
 'temperatura_maxima_na_hora_ant_aut_degc_', 'temperatura_minima_na_hora_ant_aut_degc_',
 'temperatura_orvalho_max_na_hora_ant_aut_degc_', 'temperatura_orvalho_min_na_hora_ant_aut_degc_',
 'umidade_rel_max_na_hora_ant_aut_pcnt_', 'umidade_rel_min_na_hora_ant_aut_pcnt_',
 'umidade_relativa_do_ar_horaria_pcnt_', 'vento_direcao_horaria_gr_deg_gr_',
 'vento_rajada_maxima_ms_', 'vento_velocidade_horaria_ms_','regiao',
 'uf', 'estacao', 'altitude']]

final_dataframe[list(final_dataframe)[1]] = final_dataframe[list(final_dataframe)[1]].str[:2]

In [95]:
# Function to remove accents and special characters, and to clean up names
def clean_name(name):
    # Remove content inside parentheses
    name = pd.Series(name).replace(r'\(.*?\)', '', regex=True).str.strip()
    # Remove content after a hyphen
    name = name.replace(r'\-.*', '', regex=True).str.strip()
    # Remove accents and special characters
    return name.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# Applying the cleaning function to the 'ESTACAO' column
final_dataframe['estacao'] = clean_name(final_dataframe['estacao'])

In [96]:
# Definir o caminho do diretório e do arquivo CSV
diretorio = 'DADOS_METEROLOGIA/DADOS_METEOROLOGICOS_TRATADOS/'
nome_arquivo = 'dados_meteorologicos_2023.parquet'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Verificar se o diretório existe. Se não, criar o diretório
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Lista das colunas que devem ser convertidas para float64
colunas_para_converter = [
 'precipitacao_total_horario_mm_',
 'pressao_atmosferica_ao_nivel_da_estacao_horaria_mb_',
 'pressao_atmosferica_max_na_hora_ant_aut_mb_',
 'pressao_atmosferica_min_na_hora_ant_aut_mb_',
 'temperatura_do_ar_bulbo_seco_horaria_degc_',
 'temperatura_do_ponto_de_orvalho_degc_',
 'temperatura_maxima_na_hora_ant_aut_degc_',
 'temperatura_minima_na_hora_ant_aut_degc_',
 'temperatura_orvalho_max_na_hora_ant_aut_degc_',
 'temperatura_orvalho_min_na_hora_ant_aut_degc_',
 'umidade_rel_max_na_hora_ant_aut_pcnt_',
 'umidade_rel_min_na_hora_ant_aut_pcnt_',
 'umidade_relativa_do_ar_horaria_pcnt_',
 'vento_direcao_horaria_gr_deg_gr_',
 'vento_rajada_maxima_ms_',
 'vento_velocidade_horaria_ms_']

# Converter cada coluna problemática para float64, substituindo vírgulas por pontos
for coluna in colunas_para_converter:
    if coluna in final_dataframe.columns:
        # Verifica se a coluna é do tipo 'object', o que geralmente indica strings em pandas
        if final_dataframe[coluna].dtype == 'object':
            final_dataframe[coluna] = final_dataframe[coluna].str.replace(',', '.')
        
        # Converte a coluna para numérico, tratando erros como 'coerce'
        final_dataframe[coluna] = pd.to_numeric(final_dataframe[coluna], errors='coerce')

In [97]:
final_dataframe[list(final_dataframe)[0]] = final_dataframe[list(final_dataframe)[0]].str.replace('/', '-')

In [98]:
final_dataframe = final_dataframe.groupby(['data', 'hora_utc', 'estacao', 'uf', 'regiao']).mean()

final_dataframe.reset_index(inplace=True)

In [99]:
# Tente salvar o DataFrame como um arquivo Parquet novamente
try:
    final_dataframe.to_parquet(caminho_completo)
    print("Arquivo Parquet salvo com sucesso.")
except Exception as e:
    print(f"Erro ao salvar o arquivo Parquet: {e}")

Arquivo Parquet salvo com sucesso.


## Aeroportos

In [66]:
# --- Carregamento dos Dados dos Aeroportos ---
# Definindo o caminho para o arquivo CSV dos glossários de aeródromos.
file_path = "dados_complementares/aerodromos_brasil.csv"

# Detectando a codificação do arquivo de aeródromos para assegurar a leitura correta dos dados.
file_encodings = detect_encoding(file_path)

# Obtendo a codificação detectada para o arquivo específico.
encoding = file_encodings[file_path]

# Lendo o arquivo CSV com a codificação apropriada.
# A codificação detectada é usada para lidar com possíveis caracteres especiais nos dados.
df_aeroportos = pd.read_csv(file_path, sep=';', encoding=encoding, skiprows=1)  # A codificação pode variar (ex: 'iso-8859-1', 'cp1252'), dependendo do arquivo.

df_aeroportos = df_aeroportos[['Código OACI', 'CIAD', 'Nome', 'Município', 'UF', 
                               'Município Servido', 'UF Servido', 'LATGEOPOINT', 'LONGEOPOINT', 
                               'Latitude', 'Longitude', 'Altitude', 'Situação']]

# Converter as colunas Latitude e Longitude para graus decimais
df_aeroportos['LATITUDE_AERO'] = df_aeroportos['Latitude'].apply(dms_to_decimal)
df_aeroportos['LONGITUDE_AERO'] = df_aeroportos['Longitude'].apply(dms_to_decimal)

# Aplicar a função a cada nome de coluna
df_aeroportos.columns = [padronizar_nome_coluna(col) for col in df_aeroportos.columns]

# Applying the cleaning function to the 'municipio' column
df_aeroportos['municipio'] = clean_name_aero(df_aeroportos['municipio'])

# Applying the cleaning function to the 'nome' column
df_aeroportos['nome'] = clean_name_aero(df_aeroportos['nome'])

# Applying the cleaning function to the 'uf' column
df_aeroportos['uf'] = clean_name_aero(df_aeroportos['uf'])

In [67]:
df_aeroportos = df_aeroportos[['codigo_oaci','ciad','nome','municipio','uf','altitude','latgeopoint','longeopoint','latitude_aero','longitude_aero']]

In [68]:
# Definir o caminho do diretório e do arquivo CSV
diretorio = 'dados_tratados'
nome_arquivo = 'dados_aeroportos_tratado.csv'
caminho_completo = os.path.join(diretorio, nome_arquivo)

# Verificar se o diretório existe. Se não, criar o diretório
if not os.path.exists(diretorio):
    os.makedirs(diretorio)

# Salvar o DataFrame no arquivo CSV
df_aeroportos.to_csv(caminho_completo, index=False)